In [ ]:
!pip install transformers --quiet 
!pip install pandas numpy nltk matplotlib gdown 

     |████████████████████████████████| 4.0 MB 4.5 MB/s 
     |████████████████████████████████| 880 kB 41.0 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.1 MB/s 


In [ ]:
import pandas as pd 
import numpy as np 
import random
import nltk 
from nltk.tokenize import word_tokenize

import transformers 
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
seed = 42 
torch.manual_seed(seed)
np.random.seed(seed) 
random.seed(seed)

## Загружаем тексты

In [ ]:
# download prozhito texts 
import gdown

url = 'https://drive.google.com/file/d/1oHAmsa6Nxb1B2sbWriJXCExETwSirtMj/view?usp=sharing' 
gdown.download(url, 'prozhito_texts.zip', fuzzy=True)
url = 'https://drive.google.com/file/d/1OKaDPoiS-JX1LRKYEB7o-p1v98nc1d_V/view?usp=sharing' 
gdown.download(url, 'lenta_texts.csv', fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1oHAmsa6Nxb1B2sbWriJXCExETwSirtMj
To: /content/prozhito_texts.zip
100%|██████████| 20.8M/20.8M [00:00<00:00, 79.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OKaDPoiS-JX1LRKYEB7o-p1v98nc1d_V
To: /content/lenta_text.csv
100%|██████████| 41.7M/41.7M [00:00<00:00, 93.8MB/s]


'lenta_text.csv'

In [ ]:
texts = [] 

### Прожито

In [ ]:
!unzip prozhito_texts.zip

In [ ]:
import glob, os

rootdir = 'prozhito_texts'
for subdir, dirs, files in os.walk(rootdir):
    for filename in files:
        if filename.endswith('.txt'): 
            path = os.path.join(subdir, filename)
            with open(path) as f: 
                sents = f.readlines() 
                sents = list(map(lambda x: x.strip(), sents))
                texts.extend([' '.join(sents)] * 5)

In [ ]:
len(texts)

80420

In [ ]:
texts[4]

'Пахали усадьбы в Семушине и у нас Юрка животника — Трактором .'

### Новости

In [ ]:
df = pd.read_csv('lenta_texts.csv') 
df.head()

,url,title,text,topic,tags,date,year
0,https://lenta.ru/news/2002/10/09/cbr/,Центробанк решил не радовать фальшивомонетчиков,Банк России не планирует в ближайший год вводи...,Экономика,Все,2002/10/09,2002
1,https://lenta.ru/news/2019/06/25/liniya/,Более 26 миллионов россиян посмотрели прямую л...,Более 26 миллионов россиян посмотрели трансляц...,NaN,Интернет,2019/06/25,2019
2,https://lenta.ru/news/2007/05/28/boxoffice/,"Третьим ""Пиратам Карибского моря"" не удалось о...","Фильму ""Пираты Карибского моря. На краю света""...",Культура,Все,2007/05/28,2007
3,https://lenta.ru/news/2009/03/13/broken/,Половину проверенных МиГ-29 признали непригодн...,Около 90 истребителей МиГ-29 забракованы в ход...,Наука и техника,Все,2009/03/13,2009
4,https://lenta.ru/news/2009/05/04/third/,По делу о бойне в Баку задержан второй человек,Азербайджанские правоохранительные органы заде...,Бывший СССР,Все,2009/05/04,2009


In [ ]:
for i in range(5): 
    texts.extend(df['text'].values)

In [ ]:
texts[-1]

'Счетная палата начала проверку Банка Москвы, входящего в пятерку крупнейших банков страны. Об этом сообщает РИА Новости со ссылкой на аудитора Счетной палаты Михаила Бесхмельницына. "Буквально в пятницу принято решение и уже разработана программа по проверке Банка Москвы, причем мы думаем провести комплексную проверку - его деятельности как банка, так и его участия в реализации программ по городу Москве, в том числе как агента", - сказал Бесхмельницын. С чем связана проверка Банка Москвы, аудитор не уточнил. Банк Москвы был создан в марте 1995 года правительством столицы. Сейчас столице напрямую принадлежит 46,48 процента акций. Еще 17,32 процента финансовой организации чиновники контролируют через группу дочерних компаний ОАО "Столичная страховая группа". Президент банка Андрей Бородин совместно с Львом Алалуевым владеют долей в 20,32 процента. Ранее российские деловые СМИ сообщали, что новое руководство Москвы начало переговоры по продаже части акций "Банка Москвы" государственной ф

In [ ]:
len(texts)

160520

### Перемешиваем тексты

In [ ]:
random.shuffle(texts) 
texts[-1]

'Утром пошли в ПК ДОМА , узнали , что сегодня будет собрание ОЗР * [ отряды * * защитников * * революции * * — * * Г . К . ] * и БОПа . Пошли к месту собрания ( что-то вроде клуба ) . Секретарь горкома ДОМА и Доз Мухамад смеялись по поводу бестолковой организации мероприятия . ОЗР пошел к ПК НДПА , там прошло построение и что-то вроде митинга . Потом все ( около 50 человек из них 13 женщин ) встали в колонну по одному и пошли снова к клубу . Мы не очень хотели проходить в зал * [ были * * свои * * дела * * — * * Г . К . ] , * но неудобно было отказываться от приглашения . За столом президиума сидел Анатолий Александрович , Аль Ахмат , командир полка * [ гарнизон * * Калата * * — * * Г . К . ] * , царандоевец . Выступил Аль Ахмат ( выступал без бумажки ) . После него выступил каменданте царандоя , потом Назмеддин . Свое выступление он закончил стихотворением об апрельской революции . После общего собрания состоялось еще собрание бойцов БОПа . Там выступал в основном Коля Носенко * [ сов

## Загружаем машины

In [ ]:
MODEL_PATH = "cointegrated/rubert-tiny"

In [ ]:
# s = 'Привет! Как ты?' 
# for word in word_tokenize(s): 
#     print(word)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH) 
model = AutoModelForMaskedLM.from_pretrained(MODEL_PATH) 

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# inputs = tokenizer(s_masked, return_tensors="pt") 
# labels = tokenizer(s, return_tensors="pt")["input_ids"]

# outputs = model(**inputs, labels=labels) 
# loss = outputs.loss 
# loss.backward()

In [ ]:
inputs = tokenizer(texts, return_tensors='pt',
                   max_length=512, truncation=True, padding='max_length') 

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    array = torch.flatten(mask_arr[i].nonzero()) 
    if not torch.sum(array): 
        zero_idx = None 
        try: 
            zero_idx = inputs.input_ids[i].tolist().index(0)
        except ValueError: 
            zero_idx = 512 

        idx = np.random.randint(zero_idx) 
        mask_arr[i][idx] = True 

    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids[56]

tensor([   101,   9110,   1984,    164,    164,    304,   7472,    878,    106,
           103,  28850,   6675,   1703,  11393,   7840,    869,    103,    898,
         14198,  70663,   2761, 109813,  11401,    156,    304,    839, 101527,
           128,    103,   4609,   8973,    103,   7853,  23198,    103,    326,
           132,  36335,   4650,   9450,  38784,    128,   4609,   1699,    103,
           128,   2739,    103,    861,  38784,   6305,  40819,    106,  29195,
          1655,   1699,    103,    128,   5247,   4609,   8152,    103,  14198,
           845,  58164,   8151,  13144,  54492,    842,    128,    103,   1699,
          3815,   1997,   2067,  30145,    132,    103,   5853,    130,  10785,
           128,    861,  38784,  15380,   1699,   8470,  12006,    132,  52275,
         19202,    851,    845,  62197,    842,   8953,    845,    103,  31918,
           861,  19555,  29255,    132,    777,  26916,    877,   2752,    845,
         50425,    103,  85529,  53542, 

### Готовим датасет

In [ ]:
class DiariesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = DiariesDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device) 
model.to(device)

In [ ]:
from torch.optim import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm.notebook import tqdm  # for our progress bar

epochs = 3
losses = [] 

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    loss_curr = []
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)

        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        loss_curr.append(loss.item())

    losses.append(loss_curr)
    model.save_pretrained(f'rubert_epoch_{epoch + 1}') 

  0%|          | 0/4021 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
losses = np.array(losses) 
plt.plot(np.mean(losses))